In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,545 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,019 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [74.2 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 https://developer.download.nvidia.com/compute/c

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-03-22 13:59:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.69MB/s    in 0.2s    

2023-03-22 13:59:03 (4.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...|2015-08-31 00:00:00|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...| 

In [8]:
df.count()

4366916

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [10]:
total_votes_df = vine_df.filter(vine_df["total_votes"] >= "20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [11]:
votes_df = vine_df.filter("(helpful_votes / total_votes) >= .50")
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2JDNM8F2P06FU|          4|            1|          1|   N|                Y|
|R3AM24QPLI28UY|          2|            1|          2|   N|                Y|
|R3B1NURKMCVAL1|          3|            2|          2|   N|                Y|
| R9BHBB06QD6TM|          1|            2|          2|   N|                Y|
| RFN9JY10X50F3|          1|            1|          1|   N|                Y|
| R1SEH88OC9KBZ|          5|            2|          2|   N|                Y|
|R2QS32P5QL3H8U|          5|            1|          1|   N|                Y|
| RDDUBX8JJKMX2|          5|            3|          4|   N|     

In [12]:
vine_program_df = votes_df.filter("vine == 'Y'")
vine_program_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RISSSMBIV0J9B|          5|            5|          6|   Y|                N|
|R391PM81J6G5JH|          5|            1|          1|   Y|                N|
| RSFQY2YR8A3AC|          4|            3|          5|   Y|                N|
|R3BQSPJ27GKZDK|          5|           10|         14|   Y|                N|
| RAFKWU1V19FXT|          5|            4|          4|   Y|                N|
|R1XB4YP0M7YQXD|          5|            1|          1|   Y|                N|
|R2GP7RXE3BFJI9|          5|            1|          1|   Y|                N|
| RRI85XFSGHF36|          5|            4|          4|   Y|                N|
| RNZAS0HQHECT4|          5|            1|          1|   Y|                N|
|R1OQIL720KJYCF|          5|            7|         10|   Y|     

In [13]:
vine_no_df = votes_df.filter("vine == 'N'")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2JDNM8F2P06FU|          4|            1|          1|   N|                Y|
|R3AM24QPLI28UY|          2|            1|          2|   N|                Y|
|R3B1NURKMCVAL1|          3|            2|          2|   N|                Y|
| R9BHBB06QD6TM|          1|            2|          2|   N|                Y|
| RFN9JY10X50F3|          1|            1|          1|   N|                Y|
| R1SEH88OC9KBZ|          5|            2|          2|   N|                Y|
|R2QS32P5QL3H8U|          5|            1|          1|   N|                Y|
| RDDUBX8JJKMX2|          5|            3|          4|   N|     

In [18]:
vine_program_df.count()

439

In [19]:
vine_no_df.count()

1078848

In [14]:
from pyspark.sql.functions import count
total_votes = vine_df.groupby("review_id").count()
total_votes.show() 

+--------------+-----+
|     review_id|count|
+--------------+-----+
|R3IWG4UG5CWRPV|    1|
| RGIGKQJ0A30GI|    1|
|R1N0JJZT2H3A2T|    1|
|R1NQDFEKE7E7FX|    1|
|R186RGCS4WOU8B|    1|
|R3HLUSFUDR03ZW|    1|
| RQDERR5TCNON2|    1|
|R3S0NG9K25H2ED|    1|
|R2ICBQ2OHK632F|    1|
| R557VBBH3506L|    1|
|  R5W2XEVMSOYR|    1|
|R281PWAW6UV0K6|    1|
|R1CFV5HRIQC73N|    1|
|R2XQ8J7YFY3GD9|    1|
|R1DUD71EPURX4B|    1|
|R3FMK14WGTRKM8|    1|
| R59JGQP6GJ58S|    1|
|R30CHPBRW3VRIV|    1|
|R26W1LMPTJK78R|    1|
| RYUXLNJDV4MOU|    1|
+--------------+-----+
only showing top 20 rows



In [15]:
row = votes_df.count()
print(f'Total number of reviews: {row}')

Total number of reviews: 1079287


In [45]:
five_star_total = votes_df.filter("star_rating == '5'").count()
print(f'Total number of 5 star reviews: {five_star_total}')

Total number of 5 star reviews: 594295


In [49]:
five_star_df = votes_df.filter("star_rating == '5'")
five_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
| R1SEH88OC9KBZ|          5|            2|          2|   N|                Y|
|R2QS32P5QL3H8U|          5|            1|          1|   N|                Y|
| RDDUBX8JJKMX2|          5|            3|          4|   N|                Y|
| R6DX3WK25STS2|          5|            1|          1|   N|                Y|
|R1722ED80EFYR4|          5|            1|          1|   N|                Y|
|  R9DDNXXQC0YT|          5|            1|          1|   N|                Y|
| RYNEQB5FJXJW4|          5|            2|          2|   N|                N|
| R3H2OHFQY8A6W|          5|            1|          1|   N|     

In [55]:
paid_five_star = five_star_df.filter("vine == 'Y'").count()
unpaid_five_star = five_star_df.filter("vine == 'N'").count()

In [56]:
print(f'Total 5 star rated paid vine reviews: {paid_five_star}')
print(f'Total 5 star rated unpaid vine reviews: {unpaid_five_star}')

Total 5 star rated paid vine reviews: 221
Total 5 star rated unpaid vine reviews: 594074


In [60]:
paid_percentage = round((paid_five_star/five_star_total)*100, 2)
print(f'The percentage of paid vine reviews is: {paid_percentage}%')

The percentage of paid vine reviews is: 0.04%


In [62]:
unpaid_percentage = round((unpaid_five_star/five_star_total)*100, 2)
print(f'The percentage of unpaid vine reviews is: {unpaid_percentage}%')

The percentage of unpaid vine reviews is: 99.96%
